In [153]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [154]:
from camera import CameraReader, display_frame
import ipywidgets as widgets
import numpy as np
import sys
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
import pywt
import pickle
import time
from scipy.cluster.vq import vq
from sklearn.cluster import KMeans
from matplotlib import animation, patches
from scipy import fftpack
from scipy.signal import fftconvolve, convolve2d
from IPython.display import display, Audio, HTML
from style import *
from collections import Counter,OrderedDict

%matplotlib notebook
plt.rcParams['image.cmap'] = 'gray'



In [155]:
def contar(matr):
    plano = np.ravel(matr.astype(np.uint8))
    return Counter(plano).most_common()

In [156]:
class Nodo:
    def __init__(self,hijo_izq,hijo_der):
        self.left = hijo_izq
        self.right = hijo_der
    def hijos(self):
        return (self.left,self.right)
    
def crear_arbol_de_Huffman(frecuencias):
    
    arbol_Hf = frecuencias
    #print(len(arbol_Hf))
    while len(arbol_Hf)>1:
        
        right,left = arbol_Hf[0],arbol_Hf[1]
        
        '''print("evaluar\n")
        print(right[0])
        print(right[1])
        print("\n")
        print(left[0])
        print(left[1])'''
        
        rm0x,rm0y = right[0],right[1]
        rm1x,rm1y = left[0],left[1]
    
        #if(len(arbol_Hf)>1):
        if(left[1]<right[1]):
            #print("aca\n")
            if(len(arbol_Hf)>2):
                aux1 = left
                aux2 = arbol_Hf[2]
                arbol_Hf[2] = right
                left = aux2
                right = aux1
            else:
                aux = left
                left = right
                right = aux
            '''print("añadir\n")
            print(right[0])
            print(right[1])
            print("\n")
            print(left[0])
            print(left[1])''' 

        '''print("añadir\n")
        print(right[0])
        print(right[1])
        print("\n")
        print(left[0])
        print(left[1])'''
        nodo = Nodo(left,right)
        arbol_Hf.insert(0,(nodo,left[1]+right[1]))
        
        
        arbol_Hf.remove((rm0x,rm0y))
        arbol_Hf.remove((rm1x,rm1y))
        
        #print(arbol_Hf)
        #arbol_Hf = sorted(arbol_Hf,key=lambda l:l[1], reverse=False)
        #print(arbol_Hf)
        
        
        #print(arbol_Hf)
    #print("\n")
    return arbol_Hf
        
def recorrer_arbol(nodo, prefijo='', codigo={}):
    #print(nodo[0][0])
    #print(nodo[0][1])
    if isinstance(nodo[0].left[0],Nodo):
        recorrer_arbol(nodo[0].left,prefijo+'1',codigo)
    else:
        codigo[str(nodo[0].left[0])]=prefijo+'1'
    if isinstance(nodo[0].right[0],Nodo):
        recorrer_arbol(nodo[0].right,prefijo+'0',codigo)
    else:
        codigo[str(nodo[0].right[0])]=prefijo+'0'
    
    return codigo
    

In [157]:
def replica(matrix,N=2):
    x = np.repeat(matrix[:,],N,axis=1)
    return np.repeat(x,N,axis=0)
def lh_replicada(matrix):
    matrix[1::2,:] = matrix[1::2,:]*-1
    return matrix
def hl_replicada(matrix):
    matrix[:,1::2] = matrix[:,1::2]*-1
    return matrix
def hh_replicada(matrix):
    matrix[::2,1::2] = matrix[::2,1::2]*-1
    matrix[1::2,::2] = matrix[1::2,::2]*-1
    return matrix

def inversa(LL,LH,HL,HH):
    LLrep = replica(LL)
    
    LHrep = lh_replicada(replica(LH))
    
    HLrep = hl_replicada(replica(HL))
    HHrep = hh_replicada(replica(HH))
    return LLrep + LHrep + HLrep + HHrep

In [158]:
def Cuantizacion(x,h):
    return (x//h)*h + (h//2)

In [196]:
def LowRow(image):
    lowRowImage = np.zeros(shape=(image.shape[0],image.shape[1]//2))
    contt = 0
    for i in range(0,image.shape[1],2):
        lowRowImage[:,contt:contt+1] = (image[:,i:i+1] + image[:,i+1:i+2])/2
        contt += 1
    
    """for i in range(0,image.shape[0]):    
        cont = 0
        for j in range(0, image.shape[1],2):
            lowRowImage[i][cont] = ((image[i][j] + image[i][j+1]) / 2)
            if(lowRowImage[i][cont] < 0.9):
                lowRowImage[i][cont] = 0;
            cont += 1"""
    return np.clip(lowRowImage,3,250)

def LowCol(image):
    lowColImage = np.zeros(shape=(image.shape[0]//2,image.shape[1]))
    contt = 0
    for i in range(0,image.shape[0],2):
        lowColImage[contt:contt+1,:] = (image[i:i+1,:] + image[i+1:i+2,:])/2
        contt += 1
    
    
    """for j in range(0,image.shape[1]):
        cont = 0
        for i in range(0, image.shape[0], 2):
            lowColImage[cont][j] = ((image[i][j] + image[i+1][j]) / 2)
            if(lowColImage[cont][j] < 0.9):
                lowColImage[cont][j] = 0;
            cont += 1"""
    return np.clip(lowColImage,3,250)

def HighRow(image):
    lowRowImage = LowRow(image)
    highRowImage = np.zeros(shape=lowRowImage.shape)
    for i in range(0,lowRowImage.shape[1]):
        highRowImage[:,i:i+1] = image[:,2*i:(2*i)+1] - lowRowImage[:,i:i+1]
        
    """for i in range(0,image.shape[0]):
        cont = 0
        for j in range(0,lowRowImage.shape[1]):
            highRowImage[i][cont] = image[i][2*j] - lowRowImage[i][j]
            if(highRowImage[i][cont] < 0.9):
                highRowImage[i][cont] = 0;
            cont += 1"""
    return np.clip(highRowImage,3,250)

def HighCol(image):
    lowColImage = LowCol(image)
    highColImage = np.zeros(shape=lowColImage.shape)
    for i in range(0,lowColImage.shape[0]):
        highColImage[i:i+1,:] = image[2*i:(2*i)+1,:] - lowColImage[i:i+1,:]
        
    """for j in range(0,image.shape[1]):
        cont = 0
        for i in range(0,lowColImage.shape[0]):
            highColImage[cont][j] = image[i*2][j] - lowColImage[i][j]
            if(highColImage[cont][j] < 0.9):
                highColImage[cont][j] = 0;
            cont += 1"""
    return np.clip(highColImage,3,250)

In [194]:
def compress(dic,content):
    res = ""
    for ch in content:
        code = dic[ch]
        res = res + code
    res = '1' + res + dic['end'] # Agregamos el caracter final y el 1 inicial
    res = res + (len(res) % 8 * '0') # Agregamos ceros para convertir en multiplo de 8
    return int(res,2)

#Guarda en un archivo los datos transformados con Huffman.
def store(k,dic,outn):
    ind=list(k.shape)
    res=[]
    outf = open(outn,'wb')
    for y in range(0,int(ind[1])):
        res=[]
        for x in range(0,int(ind[0])):
            res.append(str(int(k[x,y])))
        #print(res)
        com=compress(dic,res)
        #print("tamaño",sys.getsizeof(com))  #getsizeof retorna el tamaño en Bytes
        pickle.dump(com,outf)
    
    outf.close()
    
def decodif(dic, bitstr):
    res = []
    length = bitstr.bit_length() - 1
    if bitstr >> length != 1:
        raise Error("Corrupt file!")
    done = False
    while length > 0 and not done:
        shift = length - 1
        while True:
            #if shift == -1:
            #    shift=0
            #    break
            num = bitstr >> shift
            bitnum = bin(num)[3:]
            if bitnum not in dic.values():
                shift -= 1
                continue
            char = list(dic.keys())[list(dic.values()).index(bitnum)]
            if char == 'end':
                done = True
                break
            res.append(char)
            bitstr = bitstr - ((num - 1) << shift)
            length = shift
    return res
    
#Lee el archivo .huijse y llama a la funcion decodif para decodificar
def decode(ind,dic,outn):
    f = open(outn,'rb')
    objects = []
    while True:
            try:
                objects.append(pickle.load(f))
            except EOFError:
                break
    f.close()
    h = np.zeros((int(ind[0]),int(ind[1])))

    for y in range(0,int(ind[1])):
        jue = decodif(dic,objects[((int(ind[1])*0)+y)])
        #print(jue)
        cont = 0
        for x in range(0,int(ind[0])):
            h[x,y]=jue[cont]
            cont=cont+1
    return h
def limpiar_filtro(frame,size_y,size_x):
    '''c_y, c_x = frame.shape[0]/2,frame.shape[1]/2
    x_v = np.arange(0, frame.shape[1]); y_v = np.arange(0, frame.shape[0]);
    X, Y = np.meshgrid(x_v, y_v)
    sigma = 200
    mask = 1e-8 + np.exp(-(((X-c_x)/sigma/0.9)**2 + ((Y-c_y)/sigma/0.6)**2))'''
    frame_limpio = fftpack.fftshift(fftpack.fft2(frame))#*mask
    frame_limpio[240-size_y-1-frequency:240+size_y-1-frequency, 420-size_x+2:420+size_x+2] = 0
    frame_limpio[240-size_y-1+frequency:240+size_y+1+frequency, 420-size_x+6:420+size_x+6] = 0
    frame_ret = np.abs(fftpack.ifft2(fftpack.ifftshift(frame_limpio)))
    return frame_ret

def inverseM(k,n):
    ind=list(k.shape)
    for f in range(1,n+1):
        g=int(ind[0])
        g=g/2**(n-f)
        g=int(g)
        h=int(ind[1])/2**(n-f)
        h=int(h)
        k2=reverse(k[0:g,0:h,:])
        k[0:g,0:h,:]=k2
    return k

def play_vid(frame_list,size_y,size_x,out_n):
    count = 0
    display(out)
    for frame in cam:
        limpio = limpiar_filtro(frame,size_y,size_x)
        #scipy.misc.imsave("frame_sp/sp{0}.jpg".format(count),limpio)
        #frame_list.append(limpio.astype(np.uint8))
        #print(limpio)
        #print("\n")
        lowRow = LowRow(limpio)
        highRow = HighRow(limpio)
    
        LL = LowCol(lowRow)
        LH = HighCol(lowRow)
        HL = LowCol(highRow)
        HH = HighCol(highRow)
    
        #imagen_wv = np.zeros(shape=frame.shape)
        arriba = np.concatenate((LL,LH),axis=1)
        abajo = np.concatenate((HL,HH),axis=1)
        
        imagen_wv = np.concatenate((arriba,abajo),axis = 0).astype(np.uint8)
        
        lowRow_wv = LowRow(imagen_wv)
        highRow_wv = HighRow(imagen_wv)
    
        '''LL_wv = LowCol(lowRow_wv)
        LH_wv = HighCol(lowRow_wv)
        HL_wv = LowCol(highRow_wv)
        HH_wv = HighCol(highRow_wv)
        
        arriba2 = np.concatenate((LL_wv,LH_wv),axis=1)
        abajo2 = np.concatenate((HL_wv,HH_wv),axis=1)
        
        imagen_wv2 = np.concatenate((arriba2,abajo2),axis = 0).astype(np.uint8)'''
        
        n = 8
        #print(imagen_wv)
        #print("\n")
        imagen_wv = Cuantizacion(imagen_wv,n)
                                  
        freq_wv = contar(imagen_wv)
        freq_wv.append(('end',1))
        wv_tree = crear_arbol_de_Huffman(freq_wv[::-1])
        wv_dict = recorrer_arbol(wv_tree[0],'',{})
        #print(wv_dict)
        #print(imagen_wv)
        #print("\n")
        store(imagen_wv,wv_dict,out_n)
                                  
        h=decode(list(imagen_wv.shape),wv_dict,out_n)
                                  
        h=inversa(h[0:240,0:424],h[240:480,0:424],h[0:240,424:848],h[240:480,424:848])
        count +=1;
        
        display_frame(out, h.astype(np.uint8))
        #if(count==1):
        #    break;
    return count    

def play_sp(out,cam):
    count = 0
    for i in cam:
        im = scipy.misc.imread("frame_sp/sp{0}.jpg".format(count))
        display_frame(out,im)
        count 

In [195]:
cam = CameraReader()
vid_height, vid_width = cam.get_resolution()
print("Resolución del video: {0}x{1}p".format(vid_width, vid_height))
print("Cuadros por segundo: {0}".format(cam.get_fps()))
print()
out = widgets.Image(layout=widgets.Layout(width=str(vid_width), 
                                           height=str(vid_height)))

fig, ax = plt.subplots(2, 2, figsize=(7,4), tight_layout=True)

size_y=10
size_x=2
frequency=28
frame_list = [[]]
time1 = time.time()
name_out = "codificacion.phuijseheise"
#frame_list,
count = play_vid(frame_list,size_y,size_x,name_out)
time2 = time.time()
print("Total = ", time2 - time1, "segundos")
print(count/(time2-time1),"FPS")
#play_sp(out,cam)

#print(sys.getsizeof(frame_list[0]))

#Salidas en plot
#print("Tamaño frame comprimido: %0.2f MB" %(np.sum(nnz)*8/1e+6))
#print("{0} MB".format(sys.getsizeof(frame)/1e6))
#print("{0} MB".format(sys.getsizeof(frame_revuelto)/1e6))
#print("{0} MB".format(sys.getsizeof(frame_list)/1e6))

Iniciando lector de camara
OpenCV version: major: 4, minor: 1, subminor: 0
Resolución del video: 848x480p
Cuadros por segundo: 29.97002997002997



<IPython.core.display.Javascript object>

Image(value=b'', layout="Layout(height='480', width='848')")

KeyboardInterrupt: 

In [63]:
#fftpack.fftshift(np.log(1+np.abs(fftpack.fft2(frame.astype('float32')))))

In [ ]:
def compresion_RLE(matriz):
    plana = np.ravel(np.asarray(matriz)).tolist()
    #print(plana)
    #plana = ''.join(str(i) for i in plana)
    #print(chr(plana))
    #print(type(plana))
    comprimido = OrderedDict.fromkeys(matriz,0)
    for ch in matriz:
        comprimido[ch]+=1
    output=''
    for key,value in comprimido.iteritems():
        output += key +str(value)
    return output

In [94]:
matriz = [['4','3','45','12'],['4','4','4','4'],['4','3','3','3'],['3','3','3','3']]
print(compresion_RLE('matriz'))

AttributeError: 'collections.OrderedDict' object has no attribute 'iteritems'